In [ ]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

import numpy as np
import pandas as pd
from tqdm import tqdm, trange
from sklearn import model_selection
from fastai.tabular.all import *
from fastai.callback.wandb import *
SEED = 42
set_seed(SEED, reproducible=True)

In [ ]:
import wandb

try:
    from kaggle_secrets import UserSecretsClient
    user_secrets = UserSecretsClient()
    api_key = user_secrets.get_secret("wandb_api")
    wandb.login(key=api_key)
    anony = None
except:
    anony = "must"
    print('If you want to use your W&B account, go to Add-ons -> Secrets and provide your W&B access token. Use the Label name as wandb_api. \nGet your W&B access token from here: https://wandb.ai/authorize')

In [ ]:
train_df = pd.read_csv('../input/petfinder-pawpularity-score/train.csv')

In [ ]:
train_df.head()

In [ ]:
train_df["Pawpularity"].unique()

In [ ]:
train_df = train_df.drop("Id", axis=1)

In [ ]:
train_df.head()

In [ ]:
y_names = ['Pawpularity']
cont_names = list(train_df.columns.values)[1:-1]
cat_names = []
procs = [FillMissing]
splits = RandomSplitter(seed=SEED)(range_of(train_df))
#bs = 32

dls = TabularDataLoaders.from_df(train_df,'../input/petfinder-pawpularity-score/train.csv' , procs=procs, cat_names=cat_names, cont_names=cont_names, 
                                 y_names=y_names, bs=64)

In [ ]:
wandb.init(project='Pawpularity', 
                 job_type='Train',
                 anonymous='must')

In [ ]:
learn = tabular_learner(dls,layers=[500,200],metrics=rmse)

In [ ]:
cbs =  [WandbCallback()]

In [ ]:
learn.fit_one_cycle(100, 0.006,cbs=cbs)

In [ ]:
learn.show_results(max_n=12)

In [ ]:
test_df = pd.read_csv('../input/petfinder-pawpularity-score/test.csv')
test_df = test_df.drop("Id",axis=1)
test_dr = dls.test_dl(test_df)

In [ ]:
preds, _ = learn.get_preds(dl=test_dr)
preds = preds.squeeze(1)

In [ ]:
preds

In [ ]:
submit = pd.read_csv('../input/petfinder-pawpularity-score/sample_submission.csv')
submit['Pawpularity'] = preds
submit.head()

In [ ]:
submit.to_csv("submission.csv",index=False)